In [4]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yfk9atq1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-yfk9atq1
  Resolved https://github.com/huggingface/transformers to commit 2e24ee4dfa39cc0bc264b89edbccc373c8337086
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 25.1 MB/s eta 0:00:0000:0100:01
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9922875 sha256=f1f55f0b6a8bbb6edf0a07fd84b935374db0f1aa1d7ef442be6eb19e2949490f
  Stored in directory: /tmp/pip-ephem-wheel-cache-jkcrgknj/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Succ

In [3]:
!pip uninstall -y transformers

Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2


In [5]:
pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 52.4 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
device

'cuda'

In [7]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 6.10 GiB. GPU 0 has a total capacity of 14.74 GiB of which 5.08 GiB is free. Process 2689 has 9.66 GiB memory in use. Of the allocated memory 9.35 GiB is allocated by PyTorch, and 189.90 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import ViTModel, ViTFeatureExtractor, ViTImageProcessor
from PIL import Image
import torch

In [8]:
import numpy as np

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
image = Image.open('/kaggle/input/images2/IMG1.jpg')
print(type(np.array(image)))
print(np.array(image).shape)

<class 'numpy.ndarray'>
(614, 614, 3)


In [12]:
model = ViTModel.from_pretrained('google/vit-base-patch16-224')
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')

image = Image.open('/kaggle/input/images2/IMG1.jpg')
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()  # Image embedding

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [16]:
# Load the model and image processor
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

# Open an image and ensure it's RGB
image = Image.open('/kaggle/input/images2/IMG1.jpg')

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

# Extract image features
outputs = model(**inputs)
vector = outputs.last_hidden_state.mean(dim=1).detach().numpy()

print(vector)


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

[[-0.06432847  0.0523778   0.18656945  0.20612575 -0.10840616  0.09002342
   0.09847436  0.10438419  0.01100009  0.10592143 -0.06610611 -0.09910979
  -0.15100439  0.09057885  0.12569845  0.03268489 -0.20132649  0.04355562
  -0.06982531 -0.05790578 -0.09743076  0.02509805 -0.14576586  0.08895053
  -0.05265562  0.03161643 -0.19529468  0.0626617  -0.09965155 -0.29001278
  -0.14507376  0.22557437  0.08791335  0.00177897  0.03332391  0.18605205
  -0.13825248  0.09435226 -0.08593917 -0.12930559 -0.19642825 -0.08429247
  -0.24382956  0.17645566  0.20983358  0.01682295  0.09139233 -0.15736204
  -0.18509856  0.00943302  0.12606984  0.10259368 -0.31681657 -0.14263792
   0.0769181  -0.0265936  -0.20542534 -0.37944072 -0.21081628 -0.32027236
   0.19711845 -0.24671835 -0.10781609 -0.0421334   0.09764279 -0.18435186
  -0.12910953 -0.00065595 -0.01356702  0.14996903 -0.0297479   0.01884507
  -0.04180967 -0.0376329   0.19309418  0.28000805 -0.21920285  0.06154418
   0.15962468 -0.17970555  0.13335499 

In [17]:
vector.shape

(1, 768)

In [3]:
pip install einops

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel, AutoImageProcessor
from PIL import Image
import requests

processor = AutoImageProcessor.from_pretrained("nomic-ai/nomic-embed-vision-v1.5")
vision_model = AutoModel.from_pretrained("nomic-ai/nomic-embed-vision-v1.5", trust_remote_code=True)

image = Image.open("/kaggle/input/images2/IMG1.jpg")

inputs = processor(image, return_tensors="pt")

img_emb = vision_model(**inputs).last_hidden_state
img_embeddings = F.normalize(img_emb[:, 0], p=2, dim=1)
print(img_embeddings)

tensor([[-4.5462e-02, -6.6609e-03, -1.7633e-02, -5.9355e-02, -5.1656e-02,
         -7.1600e-04, -2.1137e-02, -6.4139e-02, -2.4770e-02, -2.1499e-02,
         -2.0925e-02, -1.9043e-02,  1.3856e-02, -1.0005e-01, -4.6762e-02,
          1.4080e-02,  3.2178e-02, -1.2627e-02,  6.2626e-02, -7.4154e-02,
         -3.8940e-02,  1.4525e-02,  1.0743e-02, -2.0500e-02, -8.4310e-03,
         -5.2803e-02,  1.0965e-02, -5.5497e-02, -5.1630e-02, -3.1386e-02,
         -1.5890e-02, -3.2528e-02, -4.3779e-02, -3.6613e-03, -4.8817e-02,
          2.6369e-04, -4.1981e-02, -5.6133e-04, -3.7271e-02, -3.6555e-02,
         -4.4310e-02, -6.2193e-02, -6.1359e-02, -9.4624e-03, -2.6119e-02,
          2.6254e-02, -4.0834e-02, -1.6405e-02, -1.8041e-02, -3.9325e-02,
         -5.4687e-02, -5.9463e-02, -1.6678e-03, -5.3135e-02, -5.5968e-02,
         -4.4107e-02, -6.1492e-02,  1.2281e-02,  1.8484e-02, -1.4581e-03,
         -4.8707e-02, -1.2518e-02, -2.2789e-02, -1.1079e-02, -2.1965e-02,
          1.7827e-02, -4.7583e-02, -5.

In [9]:
img_embeddings.shape

torch.Size([1, 768])